Natural Language Processing  
author: D.Thébault

Based on NLP Demystified (YouTube) https://nlpdemystified.org


# Part I: Fundamentals of NLP

## **Term Frequency - Inverse Document Frequency**

### **Reminders**

In [188]:
# Bag-of-Words (BOW)

import spacy

from scipy import spatial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# A corpus of sentences.
corpus = [
  "Red Bull drops hint on F1 engine.",
  "Honda exits F1, leaving F1 partner Red Bull.",
  "Hamilton eyes record eighth F1 title.",
  "Aston Martin announces sponsor."
]

# CountVectorizer takes a collection of text documents and creates 
# a matrix of token counts
vectorizer = CountVectorizer()

# The fit_transform method does two things:
# 1. It learns a vocabulary dictionary from the corpus.
# 2. It returns a matrix where each row represents a document and 
# each column represents a token

bow = vectorizer.fit_transform(corpus)

# Print the Document Term Matrix
# Per row, How much words in a document ?
print(bow.toarray())

# View features (tokens).
print(vectorizer.get_feature_names_out())

# View vocabulary dictionary. {'token' : index of the token}
#vectorizer.vocabulary_

[[0 0 1 1 0 1 0 0 1 0 1 0 0 0 1 0 0 1 0 0]
 [0 0 1 0 0 0 1 0 2 0 0 1 1 0 0 1 0 1 0 0]
 [0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0 0 1]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0]]
['announces' 'aston' 'bull' 'drops' 'eighth' 'engine' 'exits' 'eyes' 'f1'
 'hamilton' 'hint' 'honda' 'leaving' 'martin' 'on' 'partner' 'record'
 'red' 'sponsor' 'title']


In [189]:
# Binary + Add a special tokenizer in the CountVectorizer()

# As usual, we start by importing spaCy and loading a statistical model.
nlp = spacy.load('en_core_web_sm')

# Create a tokenizer callback using spaCy under the hood. Here, we tokenize
# the passed-in text and return the tokens, filtering out punctuation.
def spacy_tokenizer(doc):
  return [t.text for t in nlp(doc) if not t.is_punct]

# Binary BOW (0 absent, 1 presence of the token in the document)
vectorizer = CountVectorizer(tokenizer=spacy_tokenizer, lowercase=False, binary=True)
bow = vectorizer.fit_transform(corpus)
print(bow.toarray())

# View features (tokens).
print(vectorizer.get_feature_names_out())

[[0 1 1 0 0 0 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
 [0 1 1 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0]
 [0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1]
 [1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0]]
['Aston' 'Bull' 'F1' 'Hamilton' 'Honda' 'Martin' 'Red' 'announces' 'drops'
 'eighth' 'engine' 'exits' 'eyes' 'hint' 'leaving' 'on' 'partner' 'record'
 'sponsor' 'title']


/opt/anaconda3/envs/env/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [190]:
# Cosine similarity  

# With scipy: 
# The cosine method expects array_like inputs, so we need to generate
# arrays from our sparse matrix.
doc1_vs_doc2 = 1 - spatial.distance.cosine(bow[0].toarray()[0], bow[1].toarray()[0])
doc1_vs_doc3 = 1 - spatial.distance.cosine(bow[0].toarray()[0], bow[2].toarray()[0])
doc1_vs_doc4 = 1 - spatial.distance.cosine(bow[0].toarray()[0], bow[3].toarray()[0])

print(corpus)

print(f"Doc 1 vs Doc 2: {doc1_vs_doc2}")
print(f"Doc 1 vs Doc 3: {doc1_vs_doc3}")
print(f"Doc 1 vs Doc 4: {doc1_vs_doc4}")

# With sklearn:
# cosine_similarity can take either array-likes or sparse matrices.
print(cosine_similarity(bow))

['Red Bull drops hint on F1 engine.', 'Honda exits F1, leaving F1 partner Red Bull.', 'Hamilton eyes record eighth F1 title.', 'Aston Martin announces sponsor.']
Doc 1 vs Doc 2: 0.4285714285714286
Doc 1 vs Doc 3: 0.15430334996209194
Doc 1 vs Doc 4: 0.0
[[1.         0.42857143 0.15430335 0.        ]
 [0.42857143 1.         0.15430335 0.        ]
 [0.15430335 0.15430335 1.         0.        ]
 [0.         0.         0.         1.        ]]


In [191]:
# N-grams

vectorizer = CountVectorizer(tokenizer=spacy_tokenizer, 
                             lowercase=False, 
                             binary=True, 
                             ngram_range=(1,2))

bigrams = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names_out())
print('Number of features: {}'.format(len(vectorizer.get_feature_names_out())))
print(vectorizer.vocabulary_)

# Setting n_gram range to (2, 2) generates only bigrams.
vectorizer = CountVectorizer(tokenizer=spacy_tokenizer, lowercase=False, binary=True, ngram_range=(2,2))
bigrams = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(vectorizer.vocabulary_)

['Aston' 'Aston Martin' 'Bull' 'Bull drops' 'F1' 'F1 engine' 'F1 leaving'
 'F1 partner' 'F1 title' 'Hamilton' 'Hamilton eyes' 'Honda' 'Honda exits'
 'Martin' 'Martin announces' 'Red' 'Red Bull' 'announces'
 'announces sponsor' 'drops' 'drops hint' 'eighth' 'eighth F1' 'engine'
 'exits' 'exits F1' 'eyes' 'eyes record' 'hint' 'hint on' 'leaving'
 'leaving F1' 'on' 'on F1' 'partner' 'partner Red' 'record'
 'record eighth' 'sponsor' 'title']
Number of features: 40
{'Red': 15, 'Bull': 2, 'drops': 19, 'hint': 28, 'on': 32, 'F1': 4, 'engine': 23, 'Red Bull': 16, 'Bull drops': 3, 'drops hint': 20, 'hint on': 29, 'on F1': 33, 'F1 engine': 5, 'Honda': 11, 'exits': 24, 'leaving': 30, 'partner': 34, 'Honda exits': 12, 'exits F1': 25, 'F1 leaving': 6, 'leaving F1': 31, 'F1 partner': 7, 'partner Red': 35, 'Hamilton': 9, 'eyes': 26, 'record': 36, 'eighth': 21, 'title': 39, 'Hamilton eyes': 10, 'eyes record': 27, 'record eighth': 37, 'eighth F1': 22, 'F1 title': 8, 'Aston': 0, 'Martin': 13, 'announces

## 3. **TF-IDF**

Another bag-of-words technique still popular today.  

With the previous BOW techniques all the words have the same importance.  
Even removing stop words you can have highly frequent words but not informative.  

For instance in a corpus of sports you can have a high frequence of "play" and "games" but can be not informative.  

We will use relative frequence instead:  

How often a token appears in one document versus  
how often it appears in others documents.  

**If a word occurs frequently in one or few documents but not in the rest,  
it's likely meaningful for those documents and should  
be considered more "important".**  

In the other hand if a word occurs frequently across the corpus,  
then it should be considered less "important" and meaningful. 

A popular way to encode this degree of relative importance is:  
TF-IDF:
- Term Frequency
- Inverse Document Frequency

### 3.1. **TF-IDF formulas**

**Term Frequency** (TF):  

$tf(t,d) = f_{t, d}$  

Where $t$ is the term and $d$ the document.  

$tf("dog", Doc_1) = 10$  

$tf("dog", Doc_2) = 1000$  

But the Document could simply have much more words than Document 1 so we use the logarithm to dampen the effect.  

$tf(t,d) = log_{10}(f_{t, d} + 1)$  

- $tf(t,d) = log_{10}(0 + 1) = 0$   
- $tf(t,d) = log_{10}(100 + 1) = 2.0$   
- $tf(t,d) = log_{10}(1000 + 1) = 4.0$   

**Inverse Document Frequency** (IDF):  

$idf(t, D) = log_{10}(\frac{N}{n_t})$  

Give a term $t$ and a corpus $D$  

Where:

- $N$ is the number of documents
- $n_t$ is the number of documents $t$ appears in.

The fewer documents the term $t$ appears in, the _higher_ this IDF value will be, and the more the weight it will give to the word.  
If the term $t$ appears in every documents, the $idf = log_{10}(\frac{N}{N}) = 0$


**TF-IDF**  

TF-IDF score for a term in a document:  

$w_{t,d} = tf(t, d) \times idf(t, D)$  

$w_{t,d} = log_{10}(f_{t, d} + 1) \times log_{10}(\frac{N}{n_t})$  

The more frequently a term appears in a document $\times$ the fewer times it appears in others documents $\Rightarrow$ the higher the TF-IDF value is.

TF-IDF was so popular that many variations were created:  

e.g. the default TF-IDF vectorizer in scikit-learn  

$tf(t,d) = f_{t, d}$  

$idf(t, D) = ln(\frac{N + 1}{n_t + 1}) + 1$

### 3.2. **TF-IDF exemple**

#### **The corpus**

We'll use the 20 newsgroups dataset, which is a collection of 18,000 newsgroup posts across 20 topics.  
https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset

In [192]:
import spacy

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

By default, the fetcher retrieves the training subset of the data only.  
Here we only consider the posts from the sci.space area.

In [193]:
corpus = fetch_20newsgroups(categories=['sci.space'],remove=('footers', 'quotes'))
corpus.data[:2]

["From: henry@zoo.toronto.edu (Henry Spencer)\nSubject: Re: japanese moon landing?\nOrganization: U of Toronto Zoology\nLines: 21\n\n\nAny lunar satellite needs fuel to do regular orbit corrections, and when\nits fuel runs out it will crash within months.  The orbits of the Apollo\nmotherships changed noticeably during lunar missions lasting only a few\ndays.  It is *possible* that there are stable orbits here and there --\nthe Moon's gravitational field is poorly mapped -- but we know of none.\n\nPerturbations from Sun and Earth are relatively minor issues at low\naltitudes.  The big problem is that the Moon's own gravitational field\nis quite lumpy due to the irregular distribution of mass within the Moon.",
 'From: henry@zoo.toronto.edu (Henry Spencer)\nSubject: Re: Space Station Redesign, JSC Alternative #4\nOrganization: U of Toronto Zoology\nLines: 10\n\n\nGlad to see Griffin is spending his time on engineering rather than on\nritual purification of the language.  Pity he got stu

In [194]:
corpus = fetch_20newsgroups(categories=['sci.space'],remove=('headers', 'footers', 'quotes'))
# print the first two posts
corpus.data[:2]

["\nAny lunar satellite needs fuel to do regular orbit corrections, and when\nits fuel runs out it will crash within months.  The orbits of the Apollo\nmotherships changed noticeably during lunar missions lasting only a few\ndays.  It is *possible* that there are stable orbits here and there --\nthe Moon's gravitational field is poorly mapped -- but we know of none.\n\nPerturbations from Sun and Earth are relatively minor issues at low\naltitudes.  The big problem is that the Moon's own gravitational field\nis quite lumpy due to the irregular distribution of mass within the Moon.",
 '\nGlad to see Griffin is spending his time on engineering rather than on\nritual purification of the language.  Pity he got stuck with the turkey\nrather than one of the sensible options.']

We get back a Bunch container object containing the data as well as other information.  
https://scikit-learn.org/stable/modules/generated/sklearn.utils.Bunch.html  
The actual posts are accessed through the data attribute and is a list of strings, each one representing a post.

In [195]:
# A bunch is not structured (no fixe columns) but as a known number of rows:
len(corpus.data)

593

#### **Creating TF-IDF features**

In [196]:
# Like before, if we want to use spaCy's tokenizer, we need
# to create a callback. Remember to upgrade spaCy if you need
# to (refer to beginnning of file for commentary and instructions).

nlp = spacy.load('en_core_web_sm')

# We don't need named-entity recognition nor dependency parsing for
# this so these components are disabled. This will speed up the
# pipeline. We do need part-of-speech tagging however.

unwanted_pipes = ["ner", "parser"]

# Preprocessing

# For this exercise, we'll remove punctuation and spaces (which
# includes newlines), filter for tokens consisting of alphabetic
# characters, and return the lemma (which require POS tagging).
def spacy_tokenizer(doc):
  with nlp.disable_pipes(*unwanted_pipes):
    return [t.lemma_ for t in nlp(doc) if \
            not t.is_punct and \
            not t.is_space and \
            t.is_alpha]

In [197]:
sentence = "I am not available for that meeting at 12:30. I Hope everyone could manage its own tasks."
print(spacy_tokenizer(sentence))

['I', 'be', 'not', 'available', 'for', 'that', 'meeting', 'at', 'I', 'hope', 'everyone', 'could', 'manage', 'its', 'own', 'task']


Like the classes to create raw frequency and binary bag-of-words vectors, scikit-learn includes a similar class called TfidfVectorizer to create TF-IDF vectors from a corpus.  
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

The usage pattern is similar in that we call fit_transform on the corpus which generates the vocabulary dictionary (fit step),  
and generates the TF-IDF vectors (transform step).

In [198]:
%%time
# Use the default settings of TfidfVectorizer.
# We transform the corpus of documents into a matrix of caracteristics TF-IDF
vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)
features = vectorizer.fit_transform(corpus.data)

CPU times: user 9.76 s, sys: 168 ms, total: 9.93 s
Wall time: 10.2 s


In [199]:
# The number of unique tokens.
print(len(vectorizer.get_feature_names_out()))

9440


In [200]:
# The dimensions of our feature matrix. X rows (documents) by Y columns (tokens).
print(features.shape)

(593, 9440)


In [201]:
features.toarray()

array([[0.        , 0.03202688, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.09365575, 0.16199853, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.04044088, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.11334298, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.06820911, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [202]:
# What the encoding of the first document looks like in sparse format.
print(features[0])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 67 stored elements and shape (1, 9440)>
  Coords	Values
  (0, 426)	0.07007152202826399
  (0, 4918)	0.17756754701430466
  (0, 7281)	0.08827780957141772
  (0, 5546)	0.07463181843364929
  (0, 3300)	0.1988006719164653
  (0, 8491)	0.06551548062794398
  (0, 2372)	0.04343313069714861
  (0, 6885)	0.13560685996352737
  (0, 5882)	0.21555560387140033
  (0, 1846)	0.13560685996352737
  (0, 371)	0.10544208975368659
  (0, 9214)	0.07158981085158692
  (0, 4381)	0.07522603925785983
  (0, 7214)	0.09717716536087184
  (0, 5938)	0.06437012757347277
  (0, 4372)	0.07654889960067542
  (0, 9251)	0.059876060572569896
  (0, 1901)	0.13560685996352737
  (0, 9288)	0.19295422430071502
  (0, 5377)	0.10099775257415368
  (0, 8368)	0.20402991671500817
  (0, 5790)	0.0991335109087398
  (0, 451)	0.10452754121963853
  (0, 5403)	0.17102715751031994
  (0, 1344)	0.09036471134545682
  :	:
  (0, 6354)	0.15331700873692364
  (0, 5023)	0.12320196834845284
  (0, 1141)	0.04

In [203]:
# We use the Pandas library to visualize the matrix of TF-IDF caracteristics
# View the first rows of the vectorized doc
import pandas as pd
pd.set_option('display.max_columns', None)

pd.DataFrame(features.toarray(), columns = vectorizer.get_feature_names_out()).head()

,I,a,aan,aangegeven,aantal,aao,aarseth,aas,aavso,abandon,abbreviation,abc,abian,ability,able,abn,aboard,abort,aborted,about,above,abruptly,abs,absolute,absolutely,absorb,absorber,absorbtion,absorption,abstract,absurd,abundance,abuse,abutaha,abyss,ac,academic,academically,academy,accelerate,acceleration,accelerator,accelleration,accept,acceptable,acceptance,access,accessibility,accessible,accident,accidental,accidentally,accommodate,accompany,accompli,accomplish,accomplishment,accord,accordance,account,accounting,accredit,accredited,accretie,accretieschijven,accretion,accumulate,accuracy,accurate,accurately,acdi,acdis,ace,achieve,achievement,acid,acknowledge,acm,acme,acorde,acquaint,acquire,acrilic,acro,acronym,acronyms,across,acrosse,acrv,acrylonitrile,act,actinide,action,activate,activation,active,actively,activist,activiteiten,activity,actor,actual,actually,actuator,acuity,acutally,ad,ada,adam,adams,adapt,adaptability,adaptation,adapter,adc,add,addison,addition,additional,additionally,address,adelaide,adeo,adequate,adequately,adfrf,adjust,adjusted,adjustment,adler,admin,administration,administrative,administrator,admission,admit,adopt,adornato,adrift,adult,advance,advanced,advancement,advantage,adversary,adversely,advert,advertise,advertisement,advertiser,advertising,advice,advise,advisor,advisory,advocacy,advocate,aero,aeroastro,aerobee,aerobrake,aerobraking,aerodynamic,aerodynamical,aerodynamicist,aerokosmo,aeronautic,aeronautica,aeronautical,aerospace,aerospatiale,aert,aesthetic,aesthetically,af,afal,afb,affair,affect,affirmative,afflict,afford,affordable,affordably,afgedrukt,afgestemd,afield,afloop,afoot,afprl,afraid,africa,african,afrpl,afsluite,aft,after,afterlife,afternoon,afterthought,afterward,afterwards,again,against,age,agena,agency,agenda,agent,agenzia,aggregate,aggressive,aggressively,agian,agl,agn,ago,agrawal,agree,agreement,agressive,agricultural,agriculture,agu,ah,ahead,ahx,aia,aiaa,aid,aide,ailab,aim,aip,air,airborne,aircraft,airfix,airframe,airliner,airlock,airmail,airmasse,airplane,airport,airship,airspace,aj,aka,akerman,akira,aktueel,al,ala,alabama,alamos,alan,alas,alaska,albedo,albeit,albert,alberta,album,alcock,aldrin,alexander,alexandria,alexei,alexis,alford,alfred,algonquin,algorithm,ali,alien,align,alignment,alive,all,allan,alle,allegation,allege,alleghany,allegorically,allegory,allen,alleviate,allocate,allocation,allow,alloy,alluvium,ally,almanac,almost,alone,along,alot,aloud,alpha,alphabetical,alpo,already,also,alt,altare,altas,alter,altered,alternate,alternative,alternator,although,altimeter,altitude,altnet,alto,altogether,aluminized,aluminum,alveolus,alway,always,alyeska,am,amateur,amateurs,amatuer,amazing,ambassador,ambitious,ambitiously,amd,ame,amend,america,american,amherst,amiga,ammende,ammonium,ammos,ammunition,amngst,amoco,among,amongst,amor,amount,ample,amplifier,amro,amroc,amsat,amsterdam,amusing,amuture,an,analog,analogy,analyse,analysis,analyst,analyze,ancestor,ancestry,anchor,anchorperson,ancient,and,andere,anderman,anderson,andoya,andrew,andromeda,andsome,andy,aneutronic,angel,angeles,angle,anglo,angst,angstrom,angular,animal,anisotropy,ann,anne,annihilation,annihiliation,anniversary,announce,announcement,annoy,annoyed,annual,annually,annulus,anodise,anomalous,anomaly,anon,anonymous,another,ans,anser,ansi,answer,ant,antarctic,antarctica,antenna,antennae,anthology,anthony,anthropology,anti,anticipation,antimatter,antiproton,antiquated,antisatellite,any,anybody,anyhow,anymore,anyone,anything,anyway,anyways,anywhere,aoa,aoc,aout,apache,apart,aperture,aphelion,api,apiece,apl,aplace,apm,apoapsis,apocalyptic,apocentre,apogee,apollo,apology,apotheosis,appalachian,appalacian,apparent,apparently,appeal,appear,appearance,appel,append,appendicie,appendix,applaud,apple,applelink,applicable,applicant,application,apply,applyi,appointment,appoximately,apprach,appreciable,appreciate,appreciative,approach,appropriate,approval,approve,approx,approximate,approximately,approximation,apr,april,aproach,apt

In [204]:
pd.DataFrame(features.toarray(), columns = vectorizer.get_feature_names_out()).shape

(593, 9440)

#### **Querying the documents**

In [205]:
# View the 5th row of the columns 'lunar' & 'orbit' in the matrix of TF-IDF:
pd.DataFrame(features.toarray(), columns = vectorizer.get_feature_names_out()).loc[588:593,['lunar','orbit']]

,lunar,orbit
588,0.000000,0.0
589,0.000000,0.0
590,0.000000,0.0
591,0.000000,0.0
592,0.113452,0.0


The similarity measuring techniques we learned previously can be used here in the same way.  
In effect, we can query our data using this sequence:  

1. Transform our query using the same vocabulary from our fit step on our corpus.
2. Calculate the pairwise cosine similarities between each document in our corpus and our query.
3. Sort them in descending order by score.

In [206]:
# Transform the query into a TF-IDF vector.
query = ["lunar orbit"]

query_tfidf = vectorizer.transform(query)
print(query_tfidf.shape)

# Print TF-IDF of lunar, orbit in the query
pd.DataFrame(query_tfidf.toarray(), 
             columns = vectorizer.get_feature_names_out())[['lunar','orbit']]

(1, 9440)


,lunar,orbit
0,0.777334,0.629089


#### **Cosine similarities**

In [1]:
# Calculate the cosine similarities between the query and each document.
# We're calling flatten() here becaue cosine_similarity returns a list
# of lists and we just want a single list.
cosine_similarities = cosine_similarity(features, query_tfidf).flatten()

NameError: name 'cosine_similarity' is not defined

Now that we have our list of cosine similarities,  
we can use this utility function to return the indices  
of the top k documents with the highest cosine similarities.

In [208]:
import numpy as np

# numpy's argsort() method returns a list of *indices* that
# would sort an array:
# https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
#
# The sort is ascending, but we want the largest k cosine_similarites
# at the bottom of the sort. So we negate k, and get the last k
# entries of the indices list in reverse order. There are faster
# ways to do this using things like argpartition but this is
# more succinct.
def top_k(arr, k):
  kth_largest = (k + 1) * -1
  return np.argsort(arr)[:kth_largest:-1]

In [209]:
# So for our query above, these are the top five documents.
top_related_indices = top_k(cosine_similarities, 5)
print(top_related_indices)

[249 108   0 312 509]


In [210]:
# Let's take a look at their respective cosine similarities.
print(cosine_similarities[top_related_indices])

[0.47855355 0.4292246  0.2736328  0.19486489 0.19125175]


In [211]:
# Top match.
print(corpus.data[top_related_indices[0]])


Actually, Hiten wasn't originally intended to go into lunar orbit at all,
so it indeed didn't have much fuel on hand.  The lunar-orbit mission was
an afterthought, after Hagoromo (a tiny subsatellite deployed by Hiten
during a lunar flyby) had a transmitter failure and its proper insertion
into lunar orbit couldn't be positively confirmed.

It should be noted that the technique does have disadvantages.  It takes
a long time, and you end up with a relatively inconvenient lunar orbit.
If you want something useful like a low circular polar orbit, you do have
to plan to expend a certain amount of fuel, although it is reduced from
what you'd need for the brute-force approach.


In [212]:
# Second-best match.
print(corpus.data[top_related_indices[1]])


Their Hiten engineering-test mission spent a while in a highly eccentric
Earth orbit doing lunar flybys, and then was inserted into lunar orbit
using some very tricky gravity-assist-like maneuvering.  This meant that
it would crash on the Moon eventually, since there is no such thing as
a stable lunar orbit (as far as anyone knows), and I believe I recall
hearing recently that it was about to happen.


In [213]:
# Try a different query
query = ["satellite"]
query_tfidf = vectorizer.transform(query)

cosine_similarities = cosine_similarity(features, query_tfidf).flatten()
top_related_indices = top_k(cosine_similarities, 5)

print(top_related_indices)
print(cosine_similarities[top_related_indices])

[378 138 248 539  61]
[0.39068985 0.34073761 0.29838056 0.26242297 0.25695438]


In [214]:
print(corpus.data[top_related_indices[0]])



As an Amateur Radio operator (VHF 2metres) I like to keep up with what is 
going up (and for that matter what is coming down too).
 
In about 30 days I have learned ALOT about satellites current, future and 
past all the way back to Vanguard series and up to Astro D observatory 
(space).  I borrowed a book from the library called Weater Satellites (I 
think, it has a photo of the earth with a TIROS type satellite on it.)
 
I would like to build a model or have a large color poster of one of the 
TIROS satellites I think there are places in the USA that sell them.
ITOS is my favorite looking satellite, followed by AmSat-OSCAR 13 
(AO-13).
 
TTYL
73
Jim


So here we have the beginnings of a simple search engine but we're a far cry from competing with commercial off-the-shelf search engines, let alone Google.  

- For each query, we're scanning through our entire corpus, but in practice, you'll want to create an inverted index. Search applications such as ```Elasticsearch``` do that under the hood.
- You'd also want to evaluate the efficacy of your search using metrics like precision and recall.
- Document ranking also tends to be more sophisticated, using different ranking functions like ```Okapi BM25```.  With major search engines, ranking also involves hundreds of variables such as what the user searched for previously, what do they tend to click on, where are they physically, and on and on. These variables are part of the "secret sauce" and are closely guarded by companies.
- Beyond word presence, intent and meaning are playing a larger role.  

Information Retrieval is a huge, rich topic and beyond search, it's also key in tasks such as question-answering.

TF-IDF RECAP

- Captures relative importance of words in each document.

- Simple and efficient.

- Easy to compute similarities.

- Dampens stop words automatically.

- Still needs vocabulary overlap between query and document.

- Can still create sparse vectors.

- Still no way to handle out-of-vocabulary (OOV) words.

- Still doesn't capture relationship betweeen words.

- Moderately frequent terms may not be representative of document topic.

### 3.3. **TF-IDF Exercises**  

EXERCISE
Read up on these concepts we just mentioned if you're curious.
https://en.wikipedia.org/wiki/Inverted_index  
https://en.wikipedia.org/wiki/Precision_and_recall  
https://en.wikipedia.org/wiki/Okapi_BM25


In [ ]:
#
# EXERCISE: fetch multiple topics from the 20 newsgroups
# dataset and query them using the approach we followed.
# A list of topics can be found here:
# https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset
#
# If you're feeling ambitious, incorporate n-grams or
# look at how you can measure precision and recall.
#